# Chapter 10 – AI Ethics and Governance

This notebook explores fairness and bias in machine learning models using the
superhero dataset introduced earlier in the book. We'll analyze model behavior using
open source tools like ANOVA and SHAP to assess outcome differences and feature
influence. The dataset is loaded directly from the book’s GitHub repository. By
examining both group-level disparities and individual prediction logic, we connect
ethical concerns to practical, interpretable code.

### Listing 10-1: Testing Power Score Differences Across Races with ANOVA

This program loads superhero data, from Chapters 2-3, and applies an ANOVA test to evaluate whether
mean PCA Power Scores differ significantly across racial groups. It outputs summary
statistics and a visual distribution to support ethical analysis of model outcomes.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f_oneway

# --- Constants ---
BASE_URL = "https://opensourceai-book.github.io/code/datasets/"
CSV_FILE = "superheroes_info_powers2.csv"
DATA_URL = f"{BASE_URL}{CSV_FILE}"

# --- Load and Prepare Data ---
df = pd.read_csv(DATA_URL)
df = df.dropna(subset=['Race', 'PCA_Power_Score'])

# Normalize race names (optional: combine rare/variant ones)
df['Race'] = df['Race'].str.strip()

# Focus on most common races for clearer comparison
common_races = ['Human', 'Mutant', 'Cyborg', 'Alien', 'God / Eternal',
                'Android', 'Symbiote', 'Demi-God']
df = df[df['Race'].isin(common_races)]

# --- ANOVA Test: Compare PCA_Power_Score across races ---
race_groups = [group['PCA_Power_Score'].values
               for name, group in df.groupby('Race')]
f_stat, p_value = f_oneway(*race_groups)

# --- Race-wise Summary Stats ---
mean_scores = df.groupby('Race')['PCA_Power_Score'].mean().sort_values()

# --- Output Summary ---
print("=== Race Bias in Power Distribution ===\n")
print(f"Source: {CSV_FILE}")
print(f"Total samples (filtered): {len(df)}")
print("\nMean PCA Power Score by Race:")
print(mean_scores.round(2))
print(f"\nANOVA F-statistic: {f_stat:.2f}")
print(f"ANOVA P-value:     {p_value:.4f}")
print("\nInterpretation:")
if p_value < 0.05:
    print("- Power differences between races are statistically significant.")
else:
    print("- No significant difference; race-based power differences may be random.")

# --- Visualization: Boxplot of PCA Power Score by Race ---
plt.figure(figsize=(10, 6))
sns.boxplot(x='Race', y='PCA_Power_Score', data=df, palette="coolwarm")
plt.title("PCA Power Score Distribution by Race")
plt.xticks(rotation=45)
plt.ylabel("PCA Power Score")
plt.xlabel("Race")
plt.tight_layout()
plt.show()



### Listing 10-2: Explaining Superpower Predictions Using SHAP Values

This program trains a Random Forest model to predict PCA Power Score and uses
`SHAP` to explain feature contributions. The output ranks feature importance and shows
how individual traits, including race, influence prediction outcomes across the test set.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import shap
import matplotlib.pyplot as plt
import numpy as np

# --- Constants ---
BASE_URL = "https://opensourceai-book.github.io/code/datasets/"
CSV_FILE = "superheroes_info_powers2.csv"
DATA_URL = f"{BASE_URL}{CSV_FILE}"

# --- Load and Prepare Data ---
df = pd.read_csv(DATA_URL)
df = df.dropna(subset=['Race', 'Height', 'Weight', 'OPR', 'SDR',
                       'PCA_Power_Score'])

features = ['Race', 'Height', 'Weight', 'OPR', 'SDR']
X = df[features]
y = df['PCA_Power_Score']

# --- Preprocessing Pipeline ---
categorical_features = ['Race']
numeric_features = ['Height', 'Weight', 'OPR', 'SDR']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numeric_features)
    ]
)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# --- Train/Test Split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

model.fit(X_train, y_train)

# --- SHAP Explainer ---
X_transformed = model.named_steps['preprocessor'].transform(X_test).toarray()
feature_names = model.named_steps['preprocessor'].get_feature_names_out()

explainer = shap.Explainer(model.named_steps['regressor'], X_transformed,
                           feature_names=feature_names)
shap_values = explainer(X_transformed)


# --- Feature Importance Summary (printable) ---
mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
importance = pd.Series(mean_abs_shap, index=feature_names).sort_values(
    ascending=False)

print("=== SHAP Feature Importance (mean absolute contribution) ===\n")
for feature, score in importance.items():
    print(f"{feature:30} {score:.4f}")

# --- Show SHAP Summary Plot ---
shap.summary_plot(shap_values, plot_type="bar")


# --- Print SHAP contributions for first 5 predictions ---
print("\n=== Sample SHAP Contributions for Predictions ===\n")
X_sample = X_test.reset_index(drop=True).head(5)
shap_sample = shap_values[:5]
preds = model.predict(X_sample)

for i in range(5):
    print(f"Hero {i+1}: Predicted PCA_Power_Score = {preds[i]:.2f}")
    for j, name in enumerate(feature_names):
        contrib = shap_sample.values[i][j]
        if abs(contrib) > 0.1:
            print(f"  {name:30} SHAP: {contrib:+.3f}")
    print("-" * 60)
